In [1]:
import os

In [3]:
oai_key = os.environ.get('OPENAI_API_KEY')

In [8]:
import base64
from openai import OpenAI

client = OpenAI(api_key=oai_key)

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "uploads/image_20241206_181931.png"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "transcribe the text written in this image",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2x² + 3x - 5 = 0  \n(x + 1)(2x - 5) = 0  \nx = -1 ; x = 5/2', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [11]:
from pdf2image import convert_from_path
pages = convert_from_path('2007_1.pdf', 500)


In [21]:
size = pages[0].width, pages[0].height
page = pages[0].resize((size[0]//6, size[1]//6))

In [24]:
page.save('page0.png')

In [25]:
# Path to your image
image_path = 'page0.png'

# Getting the base64 string
base64_image = encode_image(image_path)

In [62]:
from pydantic import BaseModel

class Question(BaseModel):
    question: str 
    options: list[str]
    answer: str

class Page(BaseModel):
  questions: list[Question]

response = client.beta.chat.completions.parse(
  model="gpt-4o-mini",
  response_format=Page,
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "transcribe the text written in this image. also output the y_coord if the question starting and ending points between 0,1. output list of dicts in json format : [{'question':xxxx,'options':xxxx ,'correct_answer':xxxx, 'y_coord':(0.12, 0.36)}, {...}]. output valid json",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ]
)

print(response.choices[0])

ParsedChoice[Page](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[Page](content='{"questions":[{"question":"A resistance of 2 Ω is connected across one gap of a metre-bridge (the length of the wire is 100 cm) and an unknown resistance, greater than 2 Ω, is connected across the other gap. When these resistances are interchanged, the balance point shifts by 20 cm. Neglecting any corrections, the unknown resistance is","options":["3 Ω","4 Ω","5 Ω","6 Ω"],"answer":"6 Ω\',\'y_coord\':(0.12, 0.36)},{\\"question\\":\\"In an experiment to determine the focal length (f) of a concave mirror by the u-v method, a student places the object pin A on the principal axis at a distance x from the pole P. The student looks at the pin and its inverted image from a distance keeping his/her eye in line with PA. When the student shifts his/her eye towards left, the image appears to the right of the object pin. Then,\\\\n(A) x < f\\\\n(B) f < x < 2f\\\\n(C) x = 2f\\\\n(D) x 

In [67]:
import json
json.loads(response.choices[0].message.content)['questions'][0]

{'question': 'A resistance of 2 Ω is connected across one gap of a metre-bridge (the length of the wire is 100 cm) and an unknown resistance, greater than 2 Ω, is connected across the other gap. When these resistances are interchanged, the balance point shifts by 20 cm. Neglecting any corrections, the unknown resistance is',
 'options': ['3 Ω', '4 Ω', '5 Ω', '6 Ω'],
 'answer': '6 Ω\',\'y_coord\':(0.12, 0.36)},{"question":"In an experiment to determine the focal length (f) of a concave mirror by the u-v method, a student places the object pin A on the principal axis at a distance x from the pole P. The student looks at the pin and its inverted image from a distance keeping his/her eye in line with PA. When the student shifts his/her eye towards left, the image appears to the right of the object pin. Then,\\n(A) x < f\\n(B) f < x < 2f\\n(C) x = 2f\\n(D) x > 2f","options":["A","B","C","D"],"answer":"B","y_coord":(0.36, 0.60)},{"question":"Two particles of mass m each are tied at the ends 